## Final Project Submission

Please fill out:
* Student name: Reuben Hough
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Ahbineet Kulkarni
* Blog post URL:


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.preprocessing import scale
from numpy import random

In [2]:
from time import time
from optparse import OptionParser
from sklearn import metrics
from sklearn.utils.extmath import density
import argparse

In [3]:
train = pd.read_csv('cleaneddf.csv', low_memory=False)
test = pd.read_csv('cleantest.csv', low_memory=False)

In [4]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

In [5]:
display_all(train.head())

,id,gps_height,longitude,latitude,basin,region_code,lga,population,public_meeting,permit,extraction_type_class,management,payment,water_quality,quantity,source,source_class,waterpoint_type,status_group,year,month,selfbuilt,NaN Counts
0,69572,1390.0,34.938093,-9.856322,1,10,51,109,1,0,0,7,2,5,1,8,0,1,functional,2011,3,0,25
1,8776,1399.0,34.698766,-2.147466,4,19,103,280,1,1,0,11,0,5,2,5,1,1,functional,2013,3,0,24
2,34310,686.0,37.460664,-3.821329,5,20,108,250,1,1,0,7,4,5,1,0,1,2,functional,2013,2,1,25
3,67743,263.0,38.486161,-11.155298,7,24,87,58,1,1,5,7,0,5,0,3,0,2,non functional,2013,1,0,25
4,19728,1317.0,31.130847,-1.825359,4,17,26,0,1,1,0,1,0,5,3,5,1,1,functional,2011,7,1,24


In [6]:
train['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [7]:
# I'll now encode the status group, making sure to note
# which number refers to which status
le = LabelEncoder().fit(train['status_group'])
train['status_group'] = le.transform(train['status_group'])

It encoded "functional" as 0, "functional needs repair" as 1, and "non functional" as 2

In [8]:
y = train['status_group']
X = train.drop('status_group', axis=1)

In [9]:
# There are four times as many entries in the train set, 
# so I'll use .20 when testing classification
train.shape[0]/test.shape[0]

4.0

In [10]:
# Doing a test/train split on the training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=3)

In [11]:
# This fits the training data, then transforms
# both the training and testing data
scaler = StandardScaler()
scaledtraindata = scaler.fit_transform(X_train)
scaledtestdata = scaler.transform(X_test)

In [12]:
scaledtrain = pd.DataFrame(scaledtraindata, columns=X_train.columns)

In [13]:
# This gives a basic score for KNeighbors without changing very much
clf = KNeighborsClassifier()
clf.fit(scaledtraindata, y_train)
# clf.score(scaledtestdata, y_test)

KNeighborsClassifier()

In [14]:
from optparse import OptionParser

In [15]:
op = OptionParser()

In [16]:
op.add_option("--top10",
              action="store_true", dest="print_top10",
              help="Print ten most discriminative terms per class"
                   " for every classifier.")

<Option at 0x2011f48a3c8: --top10>

In [17]:
op.add_option("--confusion_matrix",
              action="store_true", dest="print_cm",
              help="Print the confusion matrix.")

<Option at 0x2011f48a748: --confusion_matrix>

In [18]:
op.add_option("--report",
              action="store_true", dest="print_report",
              help="Print a detailed classification report.")

<Option at 0x2011f48aa90: --report>

In [19]:
ops = argparse.ArgumentParser()

In [20]:
ops.add_argument("--top10",
              action="store_true", dest="print_top10",
              help="Print ten most discriminative terms per class"
                   " for every classifier.")

_StoreTrueAction(option_strings=['--top10'], dest='print_top10', nargs=0, const=True, default=False, type=None, choices=None, help='Print ten most discriminative terms per class for every classifier.', metavar=None)

In [21]:
ops.add_argument("--confusion_matrix",
              action="store_true", dest="print_cm",
              help="Print the confusion matrix.")

_StoreTrueAction(option_strings=['--confusion_matrix'], dest='print_cm', nargs=0, const=True, default=False, type=None, choices=None, help='Print the confusion matrix.', metavar=None)

In [22]:
ops.add_argument("--report",
              action="store_true", dest="print_report",
              help="Print a detailed classification report.")

_StoreTrueAction(option_strings=['--report'], dest='print_report', nargs=0, const=True, default=False, type=None, choices=None, help='Print a detailed classification report.', metavar=None)

In [23]:
import sys

In [24]:
def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')

In [25]:
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)

In [26]:
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    if opts.print_report:
        print("classification report:")
        print(metrics.classification_report(y_test, pred,
                                            target_names=target_names))
    matrix = metrics.confusion_matrix(y_test, pred)
    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time, matrix

In [33]:
results = []
for clf, name in (
        (RidgeClassifier(), "Ridge Classifier"),
        (XGBClassifier(), "XGBoost"),
        (KNeighborsClassifier(), "K Nearest Neighbors"),
        (DecisionTreeClassifier(), "Decision Tree"),
        (RandomForestClassifier(), "Random forest")):
    print(name)
    results.append(benchmark(clf))

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifier()
train time: 0.139s
test time:  0.011s
accuracy:   0.610

XGBoost
________________________________________________________________________________
Training: 
XGBClassifier()
train time: 21.558s
test time:  0.132s
accuracy:   0.740

K Nearest Neighbors
________________________________________________________________________________
Training: 
KNeighborsClassifier()
train time: 0.905s
test time:  1.160s
accuracy:   0.503

Decision Tree
________________________________________________________________________________
Training: 
DecisionTreeClassifier()
train time: 2.528s
test time:  0.031s
accuracy:   0.734

Random forest
________________________________________________________________________________
Training: 
RandomForestClassifier()
train time: 27.761s
test time:  0.564s
accuracy:   0.804



In [34]:
results[-1]

('RandomForestClassifier',
 0.8043771043771044,
 27.76079797744751,
 0.5644919872283936,
 array([[5666,  174,  581],
        [ 448,  301,  123],
        [ 932,   66, 3589]], dtype=int64))

In [ ]:
fdfgdfdf

In [30]:
results = [[x[i] for x in results] for i in range(4)]

In [32]:
results

[['RidgeClassifier',
  'XGBClassifier',
  'KNeighborsClassifier',
  'DecisionTreeClassifier',
  'RandomForestClassifier'],
 [0.6104377104377104,
  0.7396464646464647,
  0.5034511784511785,
  0.7344276094276094,
  0.8057239057239057],
 [0.08776330947875977,
  15.010863304138184,
  0.9727561473846436,
  1.1783618927001953,
  14.43410849571228],
 [0.005984783172607422,
  0.1481151580810547,
  1.599487066268921,
  0.009974002838134766,
  0.7305624485015869]]

In [ ]:
indices = np.arange(len(results) - 1)

results = [[x[i] for x in results] for i in range(4)]

In [ ]:
indices

In [ ]:
for x in results[1:len(results)]:
    print (x)

In [ ]:
from numpy import random

In [ ]:
x=random.randint(100, size=(5))

In [ ]:
normalized = x.tolist()

In [ ]:
type(x)

In [ ]:
results = scale(results[1])

In [ ]:
normalized = results[1:]

In [ ]:
normalized

In [ ]:
sdfsdfsfsd

In [ ]:
normalized = scale(x)

In [ ]:
for x in results[1:]:
    results[x] = scale(x)

In [ ]:
clf_names, score, train_time, test_time = [[x for x in results[i]] for i in range(4)]

In [ ]:
indices

In [ ]:
[x for x in results[0]]

In [ ]:
plt.figure(figsize=(12, 8))
plt.title("Score")
plt.barh(indices, score, .2, label="score", color='navy')
plt.barh(indices + .6, test_time, .2, label="test time", color='darkorange')
plt.yticks(())
plt.legend(loc='best')
plt.subplots_adjust(left=.25)
plt.subplots_adjust(top=.95)
plt.subplots_adjust(bottom=.05)

for a, b in zip(indices, clf_names):
    plt.text(-.3, a, b)

plt.show()

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhatGS)
np.set_printoptions(precision=2)

print (classification_report(y_test, yhatGS))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Functional',
                                           'Functional needs repair', 
                                           'Non functional'],
                                            normalize= False,
                                            title='Confusion matrix')